<a href="https://colab.research.google.com/github/alejandro-echaniz/tamid-x-bloomap-f22/blob/main/web_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
# List of the URL's for the different types of plants
urls = ['', 'https://www.gardenia.net/plants/plant-types/bulbs?page=', 'https://www.gardenia.net/plants/plant-types/cactus-succulents?page=',
        'https://www.gardenia.net/plants/plant-types/climbers?page=', 'https://www.gardenia.net/plants/plant-types/conifers?page=', 'https://www.gardenia.net/plants/plant-types/ferns?page=',
        'https://www.gardenia.net/plants/plant-types/fruit?page=', 'https://www.gardenia.net/plants/plant-types/herbs?page=', 'https://www.gardenia.net/plants/plant-types/ornamental-grasses?page=',
        'https://www.gardenia.net/plants/plant-types/perennials?page=', 'https://www.gardenia.net/plants/plant-types/roses?page=', 'https://www.gardenia.net/plants/plant-types/shrubs?page=',
        'https://www.gardenia.net/plants/plant-types/trees?page=', 'https://www.gardenia.net/plants/plant-types/palms-cycads?page=', 'https://www.gardenia.net/plants/plant-types/bamboos?page=',
        'https://www.gardenia.net/plants/plant-types/aquatic?page=', 'https://www.gardenia.net/plants/plant-types/orchids?page=']

# Total number of pages corresponding to the urls array
total_pages = [44, 94, 23, 29, 17, 6, 14, 9, 12, 289, 19, 136, 56, 3, 2, 16, 7]
plant_urls = set()

In [ ]:
def web_requests():
  for i, l in enumerate(urls):
    for page in tqdm(range(total_pages[i]), position=0, leave=True):
      url = l + str(page+1) # Helps us get each page number corresponding to each link
      r = requests.get(url) # sending a request to access information

      soup = bs(r.text, "html.parser") # getting the html text at that particular page

      read_urls = soup.find_all('a', string="Read More") # Accessing plant links
      for u in read_urls:
        plant_urls.add(u['href'])

In [ ]:
web_requests()
print(len(plant_urls))

100%|██████████| 7/7 [00:07<00:00,  1.01s/it]

8962


In [ ]:
def process():
  plant_df = pd.DataFrame()
  for i, l in tqdm(enumerate(plant_urls), position=0, leave=True):
    r = requests.get(l)
    table = pd.read_html(r.text)[0]
    df = pd.DataFrame(table[1]).transpose()
    df.columns = table[0]

    name = l.split("/")
    name = name[len(name) - 1]
    df['Name'] = name

    plant_df = plant_df.append(df)
  plant_df = plant_df.set_index('Name')
  return plant_df

In [ ]:
plant_df = process()

8962it [47:36,  3.14it/s]


In [ ]:
plant_df.to_csv('plant_list(uncleaned).csv')

In [ ]:
final_plant_list = plant_df.drop(plant_df.columns[[17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]],axis = 1)
final_plant_list.to_csv('plant_list(cleaned).csv')
final_plant_list

,Plant Type,Plant Family,Exposure,Season of Interest,Height,Spread,Spacing,Water Needs,Maintenance,Soil Type,Soil pH,Soil Drainage,Characteristics,Attracts,Garden Uses,Tolerance,Garden Styles
Name,,,,,,,,,,,,,,,,,
solenostemon-trusty-rusty,"Annuals, Perennials",Solenostemon - Coleus,"Full Sun, Partial Sun","Spring (Early,Mid,Late)Summer (Early,Mid,Late)...",1' – 3' (30cm – 90cm),1' – 2' (30cm – 60cm),"16"" – 20"" (40cm – 50cm)",Average,Low,"Chalk, Clay, Loam, Sand","Acid, Alkaline, Neutral",Moist but Well-Drained,"Plant of Merit, Showy, Evergreen","Birds, Hummingbirds","Beds and Borders, Patio and Containers",NaN,NaN
zinnia-giant-violet-queen,Annuals,Zinnia,Full Sun,"Summer (Mid,Late)Fall",2' – 3' (60cm – 90cm),1' – 2' (30cm – 60cm),"15"" – 18"" (37cm – 45cm)",Average,Low,"Chalk, Clay, Loam, Sand","Acid, Alkaline, Neutral","Moist but Well-Drained, Well-Drained","Cut Flowers, Showy","Birds, Butterflies, Hummingbirds","Beds and Borders, Patio and Containers",Drought,"City and Courtyard, Cutting Garden, Informal a..."
hosta-island-breeze,Perennials,Hosta - Plantain Lilies,"Partial Sun, Shade","Spring (Mid,Late)Summer (Early,Mid,Late)Fall","10"" – 1' (25cm – 30cm)",1' – 2' (30cm – 60cm),"18"" (45cm)",Average,Low,"Chalk, Clay, Loam, Sand","Acid, Alkaline, Neutral",Moist but Well-Drained,Showy,NaN,"Beds and Borders, Edging, Patio and Containers...",NaN,"City and Courtyard, Informal and Cottage, Trad..."
erigeron-glaucus,Perennials,Erigeron - Fleabanes,"Full Sun, Partial Sun","Spring (Mid,Late)Summer (Early,Mid,Late)","6"" – 1' (15cm – 30cm)",1' – 2' (30cm – 60cm),"24"" (60cm)","Low, Average",Low,"Chalk, Loam, Sand","Acid, Alkaline, Neutral","Moist but Well-Drained, Well-Drained","Showy, Evergreen",Butterflies,"Beds and Borders, Ground Covers, Patio and Con...","Deer, Drought","City and Courtyard, Coastal Garden, Gravel and..."
rose-adelaide-d-orleans,"Climbers, Roses","Rosa - Rambling Roses, Rosa - Roses","Full Sun, Partial Sun",Spring (Late)Summer (Early),14' – 15' (4.3m – 4.5m),10' – 12' (3m – 3.6m),NaN,Average,Average,"Chalk, Clay, Loam, Sand","Acid, Alkaline, Neutral","Moist but Well-Drained, Well-Drained","Fragrant, Plant of Merit, Showy, Semi-Evergreen",Butterflies,"Arbors, Pergolas, Trellises, Wall-Side Borders...",NaN,"Informal and Cottage, Traditional Garden"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cistus-creticus,Shrubs,Cistus - Rock Roses,Full Sun,"Spring (Late)Summer (Early,Mid,Late)",2' – 5' (60cm – 150cm),2' – 3' (60cm – 90cm),"36"" (90cm)",Low,Low,"Chalk, Clay, Loam, Sand","Acid, Alkaline, Neutral",Well-Drained,"Showy, Evergreen",Butterflies,"Banks and Slopes, Beds and Borders, Ground Cov...","Deer, Drought, Dry Soil, Salt","City and Courtyard, Coastal Garden, Gravel and..."
rose-rainbow-knock-out,Roses,"Rosa - Shrub Roses, Rosa - Roses","Full Sun, Partial Sun","Spring (Late)Summer (Early,Mid,Late)Fall",3' – 4' (90cm – 120cm),3' – 4' (90cm – 120cm),"36"" – 48"" (90cm – 120cm)",Average,Low,"Chalk, Clay, Loam, Sand","Acid, Alkaline, Neutral","Moist but Well-Drained, Well-Drained","Plant of Merit, Showy, Fruit & Berries",Butterflies,"Beds and Borders, Hedges and Screens",NaN,"City and Courtyard, Informal and Cottage, Trad..."
lagerstroemia-early-bird-lavender-crape-myrtle,Shrubs,Lagerstroemia - Crape Myrtles,Full Sun,"Spring (Late)Summer (Early,Mid,Late)Fall",5' – 6' (150cm – 180cm),3' – 4' (90cm – 120cm),NaN,Average,Low,"Chalk, Clay, Loam, Sand","Acid, Alkaline, Neutral",Well-Drained,Showy,NaN,"Beds and Borders, Hedges and Screens, Patio an...","Clay Soil, Deer, Drought","Coastal Garden, Informal and Cottage, Mediterr..."
